In [2]:
import sys

sys.path.append('../toolkit/')
from htmlrag import clean_html

question="When was the bellagio in las vegas built?"
html="""
<html>
<head>
<title>When was the bellagio in las vegas built?</title>
</head>
<body>
<p class="class0">The Bellagio is a luxury hotel and casino located on the Las Vegas Strip in Paradise, Nevada. It was built in 1998.</p>
</body>
<div>
<div>
<p>Some other text</p>
<p>Some other text</p>
</div>
</div>
<p class="class1"></p>
<!-- Some comment -->
<script type="text/javascript">
document.write("Hello World!");
</script>
</html>
"""

html=open("../govdata.html").read()

simplified_html = clean_html(html)
print(simplified_html)


# <html>
# <title>When was the bellagio in las vegas built?</title>
# <p>The Bellagio is a luxury hotel and casino located on the Las Vegas Strip in Paradise, Nevada. It was built in 1998.</p>
# <div>
# <p>Some other text</p>
# <p>Some other text</p>
# </div>
# </html>

 
<title>数据说话：对经济政策的预期，怎么看？</title>
<div>
<div>
<h1>
				数据说话：对经济政策的预期，怎么看？
			</h1>
<div>
<span>玉渊谭天</span>
<span>2024年12月12日 16:26</span>
</div>
</div>
<span>正在加载</span>
<div>
<p>　　今年，全球贸易保护主义逆潮不断，一些国家和地区的局势变化出乎人们的预料。</p><p>　　放眼全球，在今年这样的国际局势下，我们沉着应变、综合施策，实现了经济运行总体平稳、稳中有进，这样的成果来之不易。</p><p>　　回头看自己，今年，中国经济运行出现一些新的情况和问题，政策出台的频率与力度都很大。</p><p>　　<span>但对于不少人来说，即便“考题”再难，大家还是会对“好学生”抱有很高的期待。</span></p><p>　　这种期待，在今年，转化成为了对政策的关注——谭主利用大数据手段对2024年中国经济相关讨论进行分析后发现，“政策”是2024年舆论场最为关心的话题。</p><p>　　这些政策，瞄准的是中国经济当前面临的外部冲击、有效需求不足、投资增长存在制约等问题。</p><p>　　当政策出台后，政策能否有效，是大家最为关心的问题。</p><p>　　这些问题，怎么看？</p><p>　　今年年初，中国经济开局，就面临着很大的外部挑战。</p><p>　　美西方国家不光在国际舆论场炒作“中国经济见顶论”，还实质性地升级贸易保护主义的措施。</p><p>　　国际舆论的抹黑，加上美欧所谓的各种“调查”和关税，目标，对准的就是中国正在酝酿培育的新质生产力。</p><p>　　这只是百年未有之大变局下，博弈加剧的一个缩影，贸易，尤其成为了以美国为首的个别西方国家“武器化”的工具。</p><p>　　这样的情形之下，贸易保护主义抬头，不少国家，也主动挑起对华贸易摩擦。到11月份，部分国家就对华发起了171起贸易壁垒调查，数量远超去年。</p><p>　　今年一季度，舆论场有关中国经济的讨论中，就出现了相关的问题：</p><p>　　面临复杂多变的局势，外贸如何重返稳定发展的轨道？</p><p>　　中国经济，用数据给出了答案。</p><p>　　今年上半年，新能源汽车出口同比增长13.2%，到今天，新能源汽车国内渗透率在

In [3]:
from htmlrag import build_block_tree

# if you have mutiple HTML documents, merge them using the following code
# simplified_html = "\n".join([clean_html(html) for html in htmls])
# block_tree, simplified_html=build_block_tree(simplified_html, max_node_words=256)
block_tree, simplified_html=build_block_tree(simplified_html, max_node_words=256, zh_char=True) # for Chinese text
for block in block_tree:
    print("Block Content: ", block[0])
    print("Block Path: ", block[1])
    print("Is Leaf: ", block[2])
    print("")

# Block Content:  <title>When was the bellagio in las vegas built?</title>
# Block Path:  ['html', 'title']
# Is Leaf:  True
# 
# Block Content:  <div>
# <p>Some other text</p>
# <p>Some other text</p>
# </div>
# Block Path:  ['html', 'div']
# Is Leaf:  True
# 
# Block Content:  <p>The Bellagio is a luxury hotel and casino located on the Las Vegas Strip in Paradise, Nevada. It was built in 1998.</p>
# Block Path:  ['html', 'p']
# Is Leaf:  True


Block Content:  <title>数据说话：对经济政策的预期，怎么看？</title>
Block Path:  ['title']
Is Leaf:  True

Block Content:  <div0>
<h1>
				数据说话：对经济政策的预期，怎么看？
			</h1>
<div>
<span>玉渊谭天</span>
<span>2024年12月12日 16:26</span>
</div>
</div0>
Block Path:  ['div', 'div0']
Is Leaf:  True

Block Content:  <span>正在加载</span>
Block Path:  ['div', 'span']
Is Leaf:  True

Block Content:  <p>扫二维码 访问央视网</p>
Block Path:  ['div', 'p']
Is Leaf:  True

Block Content:  <div2>分享至：</div2>
Block Path:  ['div', 'div2']
Is Leaf:  True

Block Content:  <p0>　　今年，全球贸易保护主义逆潮不断，一些国家和地区的局势变化出乎人们的预料。</p0>
Block Path:  ['div', 'div1', 'p0']
Is Leaf:  True

Block Content:  <p1>　　放眼全球，在今年这样的国际局势下，我们沉着应变、综合施策，实现了经济运行总体平稳、稳中有进，这样的成果来之不易。</p1>
Block Path:  ['div', 'div1', 'p1']
Is Leaf:  True

Block Content:  <p2>　　回头看自己，今年，中国经济运行出现一些新的情况和问题，政策出台的频率与力度都很大。</p2>
Block Path:  ['div', 'div1', 'p2']
Is Leaf:  True

Block Content:  <p3>　　<span>但对于不少人来说，即便“考题”再难，大家还是会对“好学生”抱有很高的期待。</span></p3>
Block Path:  ['div', 'div1', 'p3']
Is Leaf:  True

Blo

In [3]:
from htmlrag import EmbedHTMLPruner

embed_model="/train_data_load/tjj_wst_hf/tjj_hf/bge-large-en/"
query_instruction_for_retrieval = "Instruct: Given a web search query, retrieve relevant passages that answer the query\nQuery: "
embed_html_pruner = EmbedHTMLPruner(embed_model=embed_model, local_inference=True, query_instruction_for_retrieval = query_instruction_for_retrieval)
# alternatively you can init a remote TEI model, refer to https://github.com/huggingface/text-embeddings-inference.
# tei_endpoint="http://YOUR_TEI_ENDPOINT"
# embed_html_pruner = EmbedHTMLPruner(embed_model=embed_model, local_inference=False, query_instruction_for_retrieval = query_instruction_for_retrieval, endpoint=tei_endpoint)
block_rankings=embed_html_pruner.calculate_block_rankings(question, simplified_html, block_tree)
print(block_rankings)

# [0, 2, 1]

/data_train/code/biz/jiejuntan/anaconda3/envs/htmlrag/lib/python3.9/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


[0, 2, 1]


In [4]:
#. alternatively you can use bm25 to rank the blocks
from htmlrag import BM25HTMLPruner
bm25_html_pruner = BM25HTMLPruner()
block_rankings=bm25_html_pruner.calculate_block_rankings(question, simplified_html, block_tree)
print(block_rankings)
assert len(block_rankings)==len(block_tree), f"The number of block rankings {len(block_rankings)} should be equal to the number of blocks {len(block_tree)}"
# [0, 2, 1]

[83, 30, 22, 23, 24, 25, 26, 27, 28, 29, 31, 82, 32, 33, 34, 35, 36, 37, 38, 39, 21, 20, 19, 18, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 40, 41, 42, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 64, 62, 43, 61, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 0]


In [5]:
from transformers import AutoTokenizer
chat_tokenizer=AutoTokenizer.from_pretrained("../../../huggingface/Meta-Llama-3.1-70B-Instruct")

max_context_window=1024
# pruned_html=embed_html_pruner.prune_HTML(simplified_html, block_tree, block_rankings, chat_tokenizer, max_context_window)
pruned_html=bm25_html_pruner.prune_HTML(simplified_html, block_tree, block_rankings, chat_tokenizer, max_context_window)
print(pruned_html)

# <html>
# <title>When was the bellagio in las vegas built?</title>
# <p>The Bellagio is a luxury hotel and casino located on the Las Vegas Strip in Paradise, Nevada. It was built in 1998.</p>
# </html>

<title>数据说话：对经济政策的预期，怎么看？</title>
<div>
<div0>
<h1>
				数据说话：对经济政策的预期，怎么看？
			</h1>
<div>
<span>玉渊谭天</span>
<span>2024年12月12日 16:26</span>
</div>
</div0>
<span>正在加载</span>
<div1>
<p0>　　今年，全球贸易保护主义逆潮不断，一些国家和地区的局势变化出乎人们的预料。</p0><p1>　　放眼全球，在今年这样的国际局势下，我们沉着应变、综合施策，实现了经济运行总体平稳、稳中有进，这样的成果来之不易。</p1><p2>　　回头看自己，今年，中国经济运行出现一些新的情况和问题，政策出台的频率与力度都很大。</p2><p3>　　<span>但对于不少人来说，即便“考题”再难，大家还是会对“好学生”抱有很高的期待。</span></p3><p4>　　这种期待，在今年，转化成为了对政策的关注——谭主利用大数据手段对2024年中国经济相关讨论进行分析后发现，“政策”是2024年舆论场最为关心的话题。</p4><p5>　　这些政策，瞄准的是中国经济当前面临的外部冲击、有效需求不足、投资增长存在制约等问题。</p5><p6>　　当政策出台后，政策能否有效，是大家最为关心的问题。</p6><p7>　　这些问题，怎么看？</p7><p8>　　今年年初，中国经济开局，就面临着很大的外部挑战。</p8><p9>　　美西方国家不光在国际舆论场炒作“中国经济见顶论”，还实质性地升级贸易保护主义的措施。</p9><p10>　　国际舆论的抹黑，加上美欧所谓的各种“调查”和关税，目标，对准的就是中国正在酝酿培育的新质生产力。</p10><p11>　　这只是百年未有之大变局下，博弈加剧的一个缩影，贸易，尤其成为了以美国为首的个别西方国家“武器化”的工具。</p11><p12>　　这样的情形之下，贸易保护主义抬头，不少国家，也主动挑起对华贸易摩擦。到11月份，部分国家就对华发起了171起贸易壁垒调查，数量远超去年。</p12><p13>　　今年一季度，舆论场有关中国经济的讨论中，就出现了相关的问题：</p13><p14>　　面临复杂多变的局势，外贸如何重返稳定发展的轨道？</p14><p15>　　中国经济，用数据给出了答案。</p1

In [6]:
block_tree, pruned_html=build_block_tree(pruned_html, max_node_words=10)
for block in block_tree:
    print("Block Content: ", block[0])
    print("Block Path: ", block[1])
    print("Is Leaf: ", block[2])
    print("")
    
# Block Content:  <title>When was the bellagio in las vegas built?</title>
# Block Path:  ['html', 'title']
# Is Leaf:  True
# 
# Block Content:  <p>The Bellagio is a luxury hotel and casino located on the Las Vegas Strip in Paradise, Nevada. It was built in 1998.</p>
# Block Path:  ['html', 'p']
# Is Leaf:  True

Block Content:  <title>数据说话：对经济政策的预期，怎么看？</title>
Block Path:  ['title']
Is Leaf:  True

Block Content:  <div0>
<h1>
				数据说话：对经济政策的预期，怎么看？
			</h1>
<div>
<span>玉渊谭天</span>
<span>2024年12月12日 16:26</span>
</div>
</div0>
Block Path:  ['div', 'div0']
Is Leaf:  True

Block Content:  <span>正在加载</span>
Block Path:  ['div', 'span']
Is Leaf:  True

Block Content:  <p0>　　今年，全球贸易保护主义逆潮不断，一些国家和地区的局势变化出乎人们的预料。</p0>
Block Path:  ['div', 'div1', 'p0']
Is Leaf:  True

Block Content:  <p1>　　放眼全球，在今年这样的国际局势下，我们沉着应变、综合施策，实现了经济运行总体平稳、稳中有进，这样的成果来之不易。</p1>
Block Path:  ['div', 'div1', 'p1']
Is Leaf:  True

Block Content:  <p2>　　回头看自己，今年，中国经济运行出现一些新的情况和问题，政策出台的频率与力度都很大。</p2>
Block Path:  ['div', 'div1', 'p2']
Is Leaf:  True

Block Content:  <p3>　　<span>但对于不少人来说，即便“考题”再难，大家还是会对“好学生”抱有很高的期待。</span></p3>
Block Path:  ['div', 'div1', 'p3']
Is Leaf:  True

Block Content:  <p4>　　这种期待，在今年，转化成为了对政策的关注——谭主利用大数据手段对2024年中国经济相关讨论进行分析后发现，“政策”是2024年舆论场最为关心的话题。</p4>
Block Path:  ['div', 'div1', 'p4']
Is Leaf:  True

Block

In [7]:
from htmlrag import GenHTMLPruner
import torch

# ckpt_path = "../../../huggingface/HTML-Pruner-Phi-3.8B"
ckpt_path = "../../../huggingface/HTML-Pruner-Llama-1B"
if torch.cuda.is_available():
    device="cuda"
else:
    device="cpu"
gen_embed_pruner = GenHTMLPruner(gen_model=ckpt_path, max_node_words=5, device=device)
block_rankings=gen_embed_pruner.calculate_block_rankings(question, pruned_html, block_tree)
print(block_rankings)

# [1, 0]

/data_train/code/biz/jiejuntan/anaconda3/envs/htmlrag/lib/python3.9/site-packages/_distutils_hack/__init__.py:54: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[1, 13, 18, 2, 9, 15, 8, 27, 10, 14, 16, 23, 24, 5, 19, 22, 11, 17, 4, 12, 6, 7, 3, 0, 25, 26, 21, 20]


In [8]:
max_context_window=32
pruned_html=gen_embed_pruner.prune_HTML(pruned_html, block_tree, block_rankings, chat_tokenizer, max_context_window)
print(pruned_html)

# <p>The Bellagio is a luxury hotel and casino located on the Las Vegas Strip in Paradise, Nevada. It was built in 1998.</p>

<title>数据说话：对经济政策的预期，怎么看？</title>
<div>
<div0>
<h1>
				数据说话：对


In [13]:
import numpy as np
list(np.argsort([1,2])[::-1])

[1, 0]